In [ ]:
import pandas as pd
import eclabfiles as ecf
from matplotlib import pyplot as plt
import numpy as np
import glob
from pickle_utils import read_ecf, read_emsi, optimise_emsi_start_voltage, alt_opt_emsi_start, read_photodiode
from config import filenames, props
%matplotlib qt

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
ecfile = glob.glob(filenames.data+'*.mpr')[-1]
datfiles = glob.glob(filenames.data+'*.dat')

In [ ]:
if glob.glob(filenames.data+'*photodiode.csv'):
    phdf, ph_start = read_photodiode(glob.glob(filenames.data+'*photodiode.csv')[0])

In [ ]:
ecfdf, ecf_start = read_ecf(ecfile)

In [ ]:
#ecf_start = 1718631832.0+5052

In [ ]:
phdf.t = phdf.t/1000+ph_start-ecf_start
phdf.photodiode = phdf.photodiode*1000

In [ ]:
plt.figure()
plt.plot(phdf.t, phdf.photodiode)
plt.plot(ecfdf.t, ecfdf.I)

In [ ]:
phdf.t = phdf.t-0.05

In [ ]:
ecfdf['photodiode'] = np.interp(ecfdf.t, phdf.t, phdf.photodiode)

In [ ]:
joined_emsi = pd.DataFrame([])
for filename in datfiles:
    emsidf, emsi_start, emsi_end = read_emsi(filename)
    if emsi_end < ecf_start:
        continue
    emsidf.loc[:, 'I'] = emsidf.I * 980
    emsi_shift = 0 #optimise_emsi_start_voltage(ecfdf, ecf_start, emsidf, emsi_start, props.voltage, props.pert_strength)
    emsidf.loc[:, 't'] = emsidf.t + emsi_start - ecf_start - emsi_shift
    joined_emsi = pd.concat([joined_emsi, emsidf], ignore_index=True)
plt.plot(joined_emsi.t, joined_emsi.I)
plt.plot(ecfdf.t, ecfdf.I)

In [ ]:
ecfdf.I = 100*ecfdf.I/props.el_area

In [ ]:
ecfdf['emsi'] = np.interp(ecfdf.t, joined_emsi.t, joined_emsi.emsi)
ecfdf['light'] = np.interp(ecfdf.t, joined_emsi.t, joined_emsi.light)

In [ ]:
ecfdf.to_pickle(filenames.data+'data.pkl')